In [1]:
#Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
#credentials
with open('/Users/csbro/.secret/yelp_api.json') as f:
    login= json.load(f)
yelp_api = YelpAPI(login['api_key'], timeout_s=5.0)

In [3]:
#Set call parameters
LOCATION='Columbus, OH'
TERM='vegan'

In [5]:

joiho ="data/in_progress_ohio.json"
JSON_FILE = joiho
JSON_FILE

'data/in_progress_ohio.json'

In [6]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [7]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[!] data/in_progress_ohio.json already exists. Deleting previous file...
[i] data/in_progress_ohio.json not found. Saving empty list to new file.
- 0 previous results found.


55

In [8]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/55 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [11]:
#load final results
final_df = pd.read_json(joiho)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,3qAXc_4kpqxEIAamgaoNpA,woodhouse-vegan-columbus,Woodhouse Vegan,https://s3-media3.fl.yelpcdn.com/bphoto/Jnwq0d...,False,https://www.yelp.com/biz/woodhouse-vegan-colum...,62,"[{'alias': 'popuprestaurants', 'title': 'Pop-U...",4.5,"{'latitude': 39.980514, 'longitude': -82.998578}","[delivery, pickup]",$$,"{'address1': '851 N 4th St', 'address2': '', '...",+16144009127,(614) 400-9127,3463.052057
1,_QEJgq3HIC5g0Rnk99RipQ,portias-cafe-columbus,Portia's Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Q0MCG7...,False,https://www.yelp.com/biz/portias-cafe-columbus...,387,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 40.0550327, 'longitude': -83.0000...","[delivery, pickup]",$$,"{'address1': '4428 Indianola Ave', 'address2':...",+16149283252,(614) 928-3252,4824.120359
2,gFnY2v0tnAr8umzHgSbk3Q,4th-and-state-columbus-2,4th & State,https://s3-media1.fl.yelpcdn.com/bphoto/J3-PkB...,False,https://www.yelp.com/biz/4th-and-state-columbu...,105,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 39.960895, 'longitude': -82.996127}","[delivery, pickup]",$$,"{'address1': '152 E State St', 'address2': '',...",+16142245461,(614) 224-5461,5648.599740
3,t82Dd-RhKZXB9eIvIH5mmQ,paulie-gees-short-north-columbus,Paulie Gee's Short North,https://s3-media3.fl.yelpcdn.com/bphoto/Bd6Pnb...,False,https://www.yelp.com/biz/paulie-gees-short-nor...,301,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.9867412256178, 'longitude': -8...","[delivery, pickup]",$$,"{'address1': '1195 N High St', 'address2': '',...",+16148080112,(614) 808-0112,2828.076443
4,cLm4oc8GIN4bkQtRKWb4Tw,zammy-s-vegan-kitchen-columbus,Zammy’s Vegan Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/B1MHxe...,False,https://www.yelp.com/biz/zammy-s-vegan-kitchen...,33,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 40.11766425508829, 'longitude': -...",[],$$,"{'address1': '1155 Worthington Woods Blvd', 'a...",+16145050407,(614) 505-0407,11832.954541


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,VLEZsm8hAuiHc68y4vyqUw,fresh-thyme-market-gahanna,Fresh Thyme Market,https://s3-media2.fl.yelpcdn.com/bphoto/njYdBo...,False,https://www.yelp.com/biz/fresh-thyme-market-ga...,43,"[{'alias': 'farmersmarket', 'title': 'Farmers ...",4.0,"{'latitude': 40.0488460933554, 'longitude': -8...",[],$$,"{'address1': '1125 N Hamilton Rd', 'address2':...",+16149341943,(614) 934-1943,12072.132486
996,DsFsaDwKf0W92ULlHhvMug,subway-marysville-33,Subway,https://s3-media1.fl.yelpcdn.com/bphoto/XMdA_1...,False,https://www.yelp.com/biz/subway-marysville-33?...,8,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.0,"{'latitude': 40.2389242614888, 'longitude': -8...",[],$,"{'address1': '1121 W 5th St', 'address2': '', ...",+19376423030,(937) 642-3030,41601.982312
997,gA1CXE2vieV3KNdvKKZ4zQ,maanas-lewis-center,Maanas,https://s3-media3.fl.yelpcdn.com/bphoto/94GVYj...,False,https://www.yelp.com/biz/maanas-lewis-center?a...,111,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",3.5,"{'latitude': 40.153572, 'longitude': -82.999481}","[pickup, delivery]",$$,"{'address1': '1335 Cameron Ave', 'address2': '...",+16144346464,(614) 434-6464,15780.314712
998,CjxfjNUxakxVcIBDk2bF8g,tropical-smoothie-cafe-columbus-5,Tropical Smoothie Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/mGl7iW...,False,https://www.yelp.com/biz/tropical-smoothie-caf...,34,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",3.5,"{'latitude': 40.146994296360845, 'longitude': ...","[pickup, delivery]",NaN,"{'address1': '8928 Lyra Dr', 'address2': None,...",+16145058689,(614) 505-8689,15191.746876
999,mBLX3LxfjJ6FqDz6XXX_0Q,zoup-columbus,Zoup!,https://s3-media1.fl.yelpcdn.com/bphoto/ka7HE-...,False,https://www.yelp.com/biz/zoup-columbus?adjust_...,44,"[{'alias': 'soup', 'title': 'Soup'}, {'alias':...",3.0,"{'latitude': 39.9610998, 'longitude': -83.0009...","[pickup, delivery]",$,"{'address1': '41 S High St', 'address2': 'Ste ...",+16144057800,(614) 405-7800,5624.191405


In [12]:
#check for duplicate results
final_df.duplicated(subset='id').sum()

0

In [15]:
final_df.to_csv('data/final_results_ohio.csv.gz', compression = 'gzip',
                index=False)